In [1]:
import os
import shutil 

import numpy as np
import tensorflow as tf

import autokeras as ak

ModuleNotFoundError: No module named 'autokeras'

In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [3]:
train_dir = 'imgs/train'
test_dir = 'imgs/test'
csv_file = 'driver_imgs_list.csv'

In [4]:
def load_images_from_folder(base_folder, size=(112, 112)):
    total_images = sum([len(files) for r, d, files in os.walk(base_folder)])

    all_images = np.empty((total_images, size[0], size[1], 3), dtype=np.uint8)
    all_labels = np.empty(total_images, dtype=np.uint8)

    idx = 0
    for folder in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder)
        
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                
                try:
                    img = Image.open(img_path).resize(size)
                    all_images[idx] = np.array(img)
                    all_labels[idx] = folder[-1]
                    idx += 1
                except IOError:
                    print(f"Error in reading {img_path}. Skipping.")

    return all_images, all_labels


In [5]:
base_folder = 'imgs/train'
images, labels = load_images_from_folder(base_folder)

In [6]:
images.shape

(22424, 112, 112, 3)

In [7]:
labels

array([2, 2, 2, ..., 4, 4, 4], dtype=uint8)

In [8]:
X = images.astype('float32') / 255

In [9]:
X.shape

(22424, 112, 112, 3)

In [10]:
Y = to_categorical(labels, 10)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [12]:
X_train.shape, X_test.shape

((17939, 112, 112, 3), (4485, 112, 112, 3))

In [13]:
y_train.shape, y_test.shape

((17939, 10), (4485, 10))

In [14]:
model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
        MaxPooling2D(2, 2),
        Flatten(),
        #Dense(128, activation='relu'),
        Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2023-12-17 22:07:32.239373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-17 22:07:32.239463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-17 22:07:32.260493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 55, 55, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 96800)             0         
                                                                 
 dense (Dense)               (None, 10)                968010    
                                                                 
Total params: 968906 (3.70 MB)
Trainable params: 968906 (3.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)

Epoch 1/20


2023-12-17 22:07:38.183168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-17 22:07:38.299873: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1438043750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-17 22:07:38.299886: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2023-12-17 22:07:38.299888: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2023-12-17 22:07:38.303631: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-17 22:07:38.358517: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


561/561 [==============================] - 3s 3ms/step - loss: 0.4747 - accuracy: 0.8815 - val_loss: 0.0497 - val_accuracy: 0.9897
Epoch 2/20
561/561 [==============================] - 1s 3ms/step - loss: 0.0202 - accuracy: 0.9962 - val_loss: 0.0394 - val_accuracy: 0.9882
Epoch 3/20
561/561 [==============================] - 2s 3ms/step - loss: 0.0088 - accuracy: 0.9982 - val_loss: 0.0234 - val_accuracy: 0.9951
Epoch 4/20
561/561 [==============================] - 1s 3ms/step - loss: 0.0087 - accuracy: 0.9979 - val_loss: 0.0208 - val_accuracy: 0.9949
Epoch 5/20
561/561 [==============================] - 1s 3ms/step - loss: 0.0015 - accuracy: 0.9997 - val_loss: 0.0104 - val_accuracy: 0.9964
Epoch 6/20
561/561 [==============================] - 2s 3ms/step - loss: 1.9311e-04 - accuracy: 1.0000 - val_loss: 0.0101 - val_accuracy: 0.9964
Epoch 7/20
561/561 [==============================] - 1s 3ms/step - loss: 1.0323e-04 - accuracy: 1.0000 - val_loss: 0.0107 - val_accuracy: 0.9962
Epoch 8/2

In [17]:
model.evaluate(X_test, y_test)

141/141 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 0.9967


[0.012591506354510784, 0.9966555237770081]

In [18]:
model.save("model.keras")

In [19]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Convert predictions from one-hot encoded to class numbers
y_pred_train_classes = np.argmax(y_pred_train, axis=1)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

# Convert true labels from one-hot encoded to class numbers
y_true_train = np.argmax(y_train, axis=1)
y_true_test = np.argmax(y_test, axis=1)

141/141 [==============================] - 0s 1ms/step


In [21]:
print(classification_report(y_true_train, y_pred_train_classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1949
           1       1.00      1.00      1.00      1809
           2       1.00      1.00      1.00      1888
           3       1.00      1.00      1.00      1854
           4       1.00      1.00      1.00      1898
           5       1.00      1.00      1.00      1836
           6       1.00      1.00      1.00      1860
           7       1.00      1.00      1.00      1605
           8       1.00      1.00      1.00      1543
           9       1.00      1.00      1.00      1697

    accuracy                           1.00     17939
   macro avg       1.00      1.00      1.00     17939
weighted avg       1.00      1.00      1.00     17939


In [26]:
print(confusion_matrix(y_true_train, y_pred_train_classes))

[[1949    0    0    0    0    0    0    0    0    0]
 [   0 1809    0    0    0    0    0    0    0    0]
 [   0    0 1888    0    0    0    0    0    0    0]
 [   0    0    0 1854    0    0    0    0    0    0]
 [   0    0    0    0 1898    0    0    0    0    0]
 [   0    0    0    0    0 1836    0    0    0    0]
 [   0    0    0    0    0    0 1860    0    0    0]
 [   0    0    0    0    0    0    0 1605    0    0]
 [   0    0    0    0    0    0    0    0 1543    0]
 [   0    0    0    0    0    0    0    0    0 1697]]


In [27]:
print(classification_report(y_true_test, y_pred_test_classes))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       540
           1       1.00      1.00      1.00       458
           2       1.00      1.00      1.00       429
           3       1.00      1.00      1.00       492
           4       1.00      1.00      1.00       428
           5       1.00      0.99      1.00       476
           6       1.00      1.00      1.00       465
           7       1.00      0.99      1.00       397
           8       1.00      1.00      1.00       368
           9       1.00      1.00      1.00       432

    accuracy                           1.00      4485
   macro avg       1.00      1.00      1.00      4485
weighted avg       1.00      1.00      1.00      4485


In [29]:
print(confusion_matrix(y_true_test, y_pred_test_classes))

[[539   0   0   0   1   0   0   0   0   0]
 [  0 457   0   0   0   0   0   0   0   1]
 [  0   0 429   0   0   0   0   0   0   0]
 [  0   0   0 492   0   0   0   0   0   0]
 [  0   0   0   2 426   0   0   0   0   0]
 [  3   0   0   0   0 472   0   1   0   0]
 [  0   0   0   0   0   0 465   0   0   0]
 [  0   0   2   0   0   0   0 395   0   0]
 [  0   0   0   0   0   0   0   0 367   1]
 [  0   0   0   0   0   0   0   0   1 431]]


In [11]:
n_folds = 10

kfold = KFold(n_folds, shuffle=True, random_state=42)

models = []
histories = []
accuracies = []
precisions = []
recalls = []
f1_scores = []
conf_matrices = []

In [14]:
input_node = ak.nodes.ImageInput()
output_node = ak.ImageBlock(
    block_type="vanilla",
    normalize=False,
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100, objective='val_accuracy' 
)

In [15]:
clf.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Trial 100 Complete [00h 00m 23s]
val_accuracy: 0.9723522663116455

Best val_accuracy So Far: 0.9723522663116455
Total elapsed time: 00h 42m 58s
Epoch 1/10
561/561 [==============================] - 4s 4ms/step - loss: 2.0660 - accuracy: 0.2216 - val_loss: 1.3987 - val_accuracy: 0.4792
Epoch 2/10
561/561 [==============================] - 2s 4ms/step - loss: 0.8549 - accuracy: 0.7088 - val_loss: 0.5662 - val_accuracy: 0.8167
Epoch 3/10
561/561 [==============================] - 2s 4ms/step - loss: 0.4477 - accuracy: 0.8609 - val_loss: 0.3390 - val_accuracy: 0.8981
Epoch 4/10
561/561 [==============================] - 2s 4ms/step - loss: 0.2928 - accuracy: 0.9115 - val_loss: 0.2669 - val_accuracy: 0.9242
Epoch 5/10
561/561 [==============================] - 2s 4ms/step - loss: 0.2179 - accuracy: 0.9357 - val_loss: 0.2197 - val_accuracy: 0.9349
Epoch 6/10
561/561 [==============================] - 2s 4ms/step - loss: 0.1588 - accuracy: 0.9553 - val_loss: 0.1979 - val_accuracy: 0.9478
Epoc

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [16]:
model = clf.export_model()

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 cast_to_float32 (CastToFlo  (None, 112, 112, 3)       0         
 at32)                                                           
                                                                 
 conv2d (Conv2D)             (None, 110, 110, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 4)       580       
                                                                 
 max_pooling2d (MaxPooling2  (None, 54, 54, 4)         0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 8)         296   